# RNN's Hyperparameters Optimization

### Setting up variables, library and data

In [0]:
import pandas as pd
import numpy as np

# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def load_data(label):
  #label = 'all'
  print("Loading data with all components")
  names_X = ['X_train_labels_niveladas.csv', 'X_test.csv']
  names_y = ['y_train_labels_niveladas.csv', 'y_test.csv']

  if label == 'all':
    features = ('fx|fy|fz|mx|my|mz')
  else:
    features = label

  X = []
  y = []

  for dataset in names_X:
    dataframe = pd.read_csv(''.join([path,dataset]), index_col=0)
    dataframe = dataframe.iloc[:, dataframe.columns.str.contains(features)]
    #X.append(np.array(dataframe))
    X.append(dataframe)

  for dataset in names_y:
    dataframe = pd.read_csv(''.join([path,dataset]),index_col=0)
    #y.append(np.array(dataframe))
    y.append(dataframe)

  print('Shape X_train: ', np.shape(X[0]))
  print('Shape X_test : ', np.shape(X[1]))
  print('Shape y_train: ', np.shape(y[0]))
  print('Shape y_test : ', np.shape(y[1]))

  #return X_train, X_test, y_train, y_test
  return X[0], X[1], y[0], y[1], features # X_train, X_test, y_train, y_test

In [0]:
def reshape_for_lstm(X_reshape):
  print("Reshaping for LSTM")
  X_reshape = np.array(X_reshape)
  #X_reshape = np.array(X_train)[:,:-1]
  number_of_features = parameters.count('|') + 1

  shape_input = int(np.shape(X_reshape)[1]/number_of_features)
  X_new = np.array([])

  for example in X_reshape:
    # split data for each component, i.e., [fx, fy, fz]
    X = np.split(example, number_of_features)
    # reshapes each component for LSTM shape
    for i, x in enumerate(X):
      X[i] = np.reshape(x, (shape_input, 1))

    # concatenate all components with new shape and transpose it to be in (n_experiment, timesteps, components)
    X_example = np.concatenate([[x for x in X]])
    X_example = np.transpose(X_example)
    if X_new.shape == (0,):
      X_new = X_example
    else:
      X_new = np.concatenate((X_new, X_example))

  print("X_new.shape = ", X_new.shape)
  #print("new y data shape = ", X_test_full.shape)

  return X_new

In [0]:
## Data path handlers
#TRAIN_TEST_SET_PATH = '../../Data/train_test/'
path = r'/content/drive/My Drive/USP/Doutorado/Artigos - Escrita/2020 - IROS/Datasets/SMOTe_Nivelado/all_components/'
META_DATA_PATH = r'/content/drive/My Drive/USP/Doutorado/Artigos - Escrita/2020 - IROS/Datasets/SMOTe_Nivelado/all_components/glahr_meta/'
AUG_SET_PATH = '../../Data/aug_data_all/'
MODEL_PATH = r'/content/drive/My Drive/USP/Doutorado/Artigos - Escrita/2020 - IROS/Datasets/SMOTe_Nivelado/all_components/glahr_models/'
IMAGES_PATH = r'/content/drive/My Drive/USP/Doutorado/Artigos - Escrita/2020 - IROS/Datasets/SMOTe_Nivelado/all_components/glahr_images/'

## Selecting the desired features
parameters = 'fx|fy|fz|mx|my|mz'

In [0]:
### Getting data and selecting features
#X_train = pd.read_csv(TRAIN_TEST_SET_PATH+'X_train.csv',index_col=0)
#y_train = pd.read_csv(TRAIN_TEST_SET_PATH+'y_train.csv',index_col=0)
#X_test = pd.read_csv(TRAIN_TEST_SET_PATH+'X_test.csv',index_col=0)
#y_test = pd.read_csv(TRAIN_TEST_SET_PATH+'y_test.csv',index_col=0)

X_train, X_test, y_train, y_test, features = load_data(parameters)
X_train['labels'] = y_train.copy()

In [0]:
### Creating the validation set
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=27)
#for train, val in split.split(X_train, X_train['labels']):
for train, val in split.split(X_train, X_train['labels']):
    X_train_vl = X_train.iloc[train].copy()
    X_val = X_train.iloc[val].copy()
    
y_train_vl = X_train_vl['labels'].copy()
y_val = X_val['labels'].copy()

X_train_vl = X_train_vl.iloc[:, ~X_train_vl.columns.str.contains('labels')]
X_val = X_val.iloc[:, ~X_val.columns.str.contains('labels')]
#X_train = X_train.iloc[:, ~X_train.columns.str.contains('labels')]

y_train_vl = np.array(y_train_vl)-1
y_val = np.array(y_val)-1
y_test = np.array(y_test)-1
y_train = np.array(y_train)-1

print("X_train_vl.shape = ", X_train_vl.shape)
print("X_val.shape = ", X_val.shape)

print("y_train_vl.shape = ", y_train_vl.shape)
print("y_val.shape = ", y_val.shape)

In [0]:
### Standardizing the data
#from sklearn.preprocessing import StandardScaler

#std_scaler = StandardScaler()

#X_train = std_scaler.fit_transform(X_train)
#X_test = std_scaler.transform(X_test)
#X_train_vl = std_scaler.transform(X_train_vl)
#X_val = std_scaler.transform(X_val)

# AJUSTAR AQUI
X_train = X_train/30
X_test = X_test/30
X_train_vl = X_train_vl/30
X_val = X_val/30
print("X_train.shape = ", X_train.shape)
print("X_test.shape = ", X_test.shape)
print("X_train_vl.shape = ", X_train_vl.shape)
print("X_val.shape = ", X_val.shape)

In [0]:
### Reshaping data for LSTM
X_train = reshape_for_lstm(np.array(X_train)[:,:-1])
X_test = reshape_for_lstm(X_test)
X_train_vl = reshape_for_lstm(X_train_vl)
X_val = reshape_for_lstm(X_val)

print("\n\n\nX_train.shape = ", X_train.shape)
print("X_test.shape = ", X_test.shape)
print("X_train_vl.shape = ", X_train_vl.shape)
print("X_val.shape = ", X_val.shape)

### Setting up the rnn model and callbacks

In [0]:
### Training a simple rnn to use as reference
import tensorflow as tf
from tensorflow import keras

In [0]:
'''
### Model sketch
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[1556]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(3, activation='softmax'))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy'
                 ,optimizer=optimizer
                 ,metrics=['accuracy'])
    return model

mlp_simple = keras.wrappers.scikit_learn.KerasClassifier(build_model)
'''

In [0]:
### Model LSTM sketch
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=1556, features=6, output_shape=3, dropout_com=0.0, dropout_rec=0.0):

  model = keras.models.Sequential()
  # input layer
  if n_hidden == 0:
    model.add(keras.layers.LSTM(units=n_neurons, input_shape=(input_shape,features), return_sequences=False, dropout=dropout_com))
  else:
    model.add(keras.layers.LSTM(units=n_neurons, input_shape=(input_shape,features), return_sequences=True, dropout=dropout_com, recurrent_dropout=dropout_rec))
    if n_hidden > 1:
      for layer in range(n_hidden-1):
        model.add(keras.layers.LSTM(units=n_neurons, return_sequences=True, dropout=dropout_com, recurrent_dropout=dropout_rec))
      else:
        model.add(keras.layers.LSTM(units=n_neurons, return_sequences=False, dropout=dropout_com))

  # output layer
  model.add(keras.layers.Dense(output_shape, activation='softmax'))
  optimizer = keras.optimizers.SGD(lr=learning_rate)
  model.compile(loss='sparse_categorical_crossentropy'
                ,optimizer=optimizer
                ,metrics=['accuracy'])
  print(model.summary())
  return model

lstm_model = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [0]:
### Setting Callbacks
early_stopping = keras.callbacks.EarlyStopping(patience=15,
                                               restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint(MODEL_PATH+"lstm_simple.h5",
                                                save_best_only=True)                                     

get_stats = pd.DataFrame([])
get_stats.to_csv(META_DATA_PATH+'lstm_error_analysis.csv')

### Hyperparameter optimization

In [0]:
### Setting parameters range for hyperparameter optimzation
### Here we will use random search as the optimization method
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV


params = {
    'n_hidden': np.arange(1,5),
    'n_neurons': np.arange(1,32),
    'learning_rate': reciprocal(3e-4, 3e-2),
    'dropout_com': np.arange(0,6)/10,
    'dropout_rec': np.arange(0,6)/10
    #'input_shape': [X_train_vl.shape[1]]
}

In [0]:
### Hyperparameter Optimization
rnd_search = RandomizedSearchCV(lstm_model, params, n_iter=10, cv=3, n_jobs=1)
print("X_train_vl.shape = ", X_train_vl.shape)
print("y_train_vl.shape = ", y_train_vl.shape)
rnd_search.fit(X_train_vl, y_train_vl, epochs=100,
               validation_split=0.15,
               callbacks=[early_stopping, checkpoint_cb])

In [0]:
### Best validation score
rnd_search.best_score_

In [0]:
### Best set of hyperparameters
best_parameters = pd.DataFrame(rnd_search.best_params_, index=['values'])
best_parameters.to_csv(MODEL_PATH+'mlp_simple_bp.csv')

rnd_search.best_params_

In [0]:
pd.read_csv(MODEL_PATH+'mlp_simple_bp.csv', usecols=['learning_rate']).iloc[0]

### Training the reference model

In [0]:
### Selecting and retraining the best model
best = rnd_search.best_estimator_.model
best.fit(X_train, y_train, epochs=200,
         validation_data=(X_test, y_test),
         callbacks=[early_stopping])

In [0]:
### Model evaluation in accuracy terms
best.evaluate(X_test, y_test)